In [ ]:
# python radix sort
def radixsort(lst):
    """
    Sorts a list of integers using Radix Sort.

    :param lst: List of integers to be sorted.
    :return: Sorted list of integers.
    """
    # Find the maximum number to determine the number of digits
    max_element = max(lst)
    max_digits = len(str(abs(max_element)))

    # Perform counting sort for every digit
    exp = 1
    while max_digits > 0:
        # Initialize the count array
        count = [0] * 10

        # Store the count of occurrences in count[]
        for num in lst:
            digit = (num // exp) % 10
            count[digit] += 1

        # Calculate cumulative count
        for i in range(1, 10):
            count[i] += count[i - 1]

        # Build the output array
        output = [0] * len(lst)
        for i in range(len(lst) - 1, -1, -1):
            digit = (lst[i] // exp) % 10
            output[count[digit] - 1] = lst[i]
            count[digit] -= 1

        # Copy the output array to lst[], so that lst[] now contains sorted numbers according to current digit
        lst = output[:]
        exp *= 10
        max_digits -= 1

    return lst

### Example Usage:
numbers = [170, 45, 75, 90, 802, 24, 2, 66,7,99,90]
print("Original List:", numbers)
sorted_numbers = radixsort(numbers)
print("Sorted List:", sorted_numbers)


In [1]:
# fortran module radix sort
import numpy as np
import ctypes

# Load the shared library
lib = ctypes.CDLL('./libradixsort.so')

# Define the argument types for the radixsort function
lib.radixsort.argtypes = [np.ctypeslib.ndpointer(dtype=np.int64, flags='C_CONTIGUOUS'),
                          ctypes.c_int]
lib.radixsort.restype = None  # The function doesn't return a value

def radixsort(arr):
    """
    Perform radix sort on the input array in-place.
    
    :param arr: Input array of 64-bit integers
    :return: None (array is sorted in-place)
    """
    arr = np.ascontiguousarray(arr, dtype=np.int64)
    n = len(arr)
    lib.radixsort(arr, n)

# Example usage
if __name__ == "__main__":
    # Create a random array of 64-bit integers
    input_array = np.random.randint(0, 1000000, size=1000000, dtype=np.int64)
    
    print("First 100 elements before sorting:", input_array[:100])
    
    # Sort the array
    radixsort(input_array)
    
    print("First 100 elements after sorting:", input_array[:100])
    
    # Verify the sort
    assert np.all(input_array[:-1] <= input_array[1:]), "Array is not sorted!"
    print("Array is correctly sorted.")

First 100 elements before sorting: [347126 800014  15136 662033 108068 189979 890152 984309 339073 450338
 696082 367722 333238 125613 277258 973340 836632 615847 526449 107248
 945576 258477 624344 710829 565395 818202 232684 901527 516296 106107
  24504 374467 361050 448088 941722 552247 225130 497039  85213 798195
 249434 320443 964359 178299 667520 834581 177427 242932 931840 899734
 761143 237494 490425   9964 530996 667395 442433 997617 527812 385644
  22757 552407  14671 805937  70500  12392 337214 518275 557271 350981
 727946 695681 703804 855113 430666 768465 296598  14338 550421 484922
 161528 800601 574878 337492 385425 384012 346594 851704 322473 889648
  12158 281486 944805 108373  40909 783371 149581  20808 685726 198019]
First 100 elements after sorting: [  0   1   1   2   2   3   4   4   7   9   9  10  11  12  13  16  16  16
  17  17  18  19  21  24  26  26  26  26  27  29  29  33  33  34  36  39
  39  41  42  43  45  45  46  46  47  48  51  51  51  51  52  53  53  54
 

In [1]:
# fortran module, numpy and python sort compared
import numpy as np
import ctypes
import time

# Load the shared library
lib = ctypes.CDLL('./libradixsort.so')

# Define the argument types for the radixsort function
lib.radixsort.argtypes = [np.ctypeslib.ndpointer(dtype=np.int64, flags='C_CONTIGUOUS'),
                          ctypes.c_int]
lib.radixsort.restype = None

def radixsort(arr):
    arr_copy = np.copy(arr)
    lib.radixsort(arr_copy, len(arr_copy))
    return arr_copy

def time_sort(sort_function, arr, name):
    arr_copy = np.copy(arr)
    start_time = time.time()
    sorted_arr = sort_function(arr_copy)
    end_time = time.time()
    print(f"{name} took {end_time - start_time:.6f} seconds")
    return sorted_arr

# bigger is better for fortran > numpy 100M and up
# 10T uses more than 80GB, try 1T
if __name__ == "__main__":
    n = 1_000_000_000
    input_array = np.random.randint(0, 1_000_000_000, size=n, dtype=np.int64)
    
    print(f"Sorting an array of {n} elements")
    print("First 10 elements before sorting:", input_array[:10])
    
    fortran_sorted = time_sort(radixsort, input_array, "Fortran Radix Sort")
    numpy_sorted = time_sort(np.sort, input_array, "NumPy Sort")
    python_sorted = time_sort(sorted, input_array, "Python Built-in Sort")
    
    print("\nFirst 10 elements after Fortran sorting:", fortran_sorted[:10])
    print("First 10 elements after NumPy sorting:", numpy_sorted[:10])
    print("First 10 elements after Python sorting:", python_sorted[:10])
    
    # Verify that all sorts produce the same result
    assert np.array_equal(fortran_sorted, numpy_sorted), "Fortran sort doesn't match NumPy sort!"
    assert np.array_equal(fortran_sorted, python_sorted), "Fortran sort doesn't match Python sort!"
    print("All sorts produce the same result.")

    # Check if arrays are actually sorted
    assert np.all(fortran_sorted[:-1] <= fortran_sorted[1:]), "Fortran array is not sorted!"
    assert np.all(numpy_sorted[:-1] <= numpy_sorted[1:]), "NumPy array is not sorted!"
    assert all(python_sorted[i] <= python_sorted[i+1] for i in range(len(python_sorted)-1)), "Python array is not sorted!"
    print("All arrays are correctly sorted.")
    # fortran and numpy took 40GB+ and 83s vs 100s for numpy Sorting an array of 1,000,000,000 elements
    # python took 83GB plus swap and 1830s!

Sorting an array of 1000000000 elements
First 10 elements before sorting: [872305779 890932924 917264515  43324407 129791628 123966711 901449954
 620116043 932781744 650655179]
Fortran Radix Sort took 83.998281 seconds
NumPy Sort took 100.484882 seconds
Python Built-in Sort took 1830.611911 seconds

First 10 elements after Fortran sorting: [2 3 3 5 7 7 8 8 8 9]
First 10 elements after NumPy sorting: [2 3 3 5 7 7 8 8 8 9]
First 10 elements after Python sorting: [2, 3, 3, 5, 7, 7, 8, 8, 8, 9]
All sorts produce the same result.
All arrays are correctly sorted.
